In [3]:
import os
import pandas as pd
from datetime import datetime

def dt(hour, minute, second=0):
    return datetime(2023, 1, 1, hour, minute, second)

# Set the environment variables for Localstack
os.environ['AWS_ACCESS_KEY_ID'] = 'test'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'test'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['S3_ENDPOINT_URL'] = 'http://localhost:4566'

# Define the S3 bucket and file paths
bucket = "nyc-duration"
input_file = f's3://{bucket}/in/2023-01.parquet'
output_file = f's3://{bucket}/out/2023-01.parquet'

# Create Localstack bucket
os.system(f"aws --endpoint-url=http://localhost:4566 s3 mb s3://{bucket}")

# Create the dataframe
data = [
    (None, None, dt(1, 1), dt(1, 10)),
    (1, 1, dt(1, 2), dt(1, 10)),
    (1, None, dt(1, 2, 0), dt(1, 2, 59)),
    (3, 4, dt(1, 2, 0), dt(2, 2, 1)),      
]
columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']
df_input = pd.DataFrame(data, columns=columns)

# Save the dataframe to S3
options = {
    'client_kwargs': {
        'endpoint_url': os.getenv("S3_ENDPOINT_URL")
    }
}

df_input.to_parquet(
    input_file,
    engine='pyarrow',
    compression=None,
    index=False,
    storage_options=options
)

# Verify the file was created
os.system(f"aws --endpoint-url=http://localhost:4566 s3 ls s3://{bucket}/in/")


make_bucket: nyc-duration
2024-07-03 22:11:57       3620 2023-01.parquet


0